In [1]:
from Sindonews import Sindonews #class news Scrapper
from datetime import timedelta, date, datetime
from IPython.display import clear_output

In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [5]:
obj = Sindonews()
end_date = date(2018, 7, 24)
start_date = date(2018, 4, 26)
init_page = 23
init_offset = 220

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, init_offset, single_date.strftime("%Y-%m-%d"))
    init_page = 1
    init_offset = 0

2018-04-30
page  1
https://index.sindonews.com/index/0/0?t=2018-04-30
https://sports.sindonews.com/read/1302067/50/diberi-harapan-palsu-petinju-inggris-ini-kecewa-berat-dengan-wbc-1525103724
Insert berita  Diberi Harapan Palsu, Petinju Inggris Ini Kecewa Berat dengan WBC
masuk
https://international.sindonews.com/read/1302065/40/presiden-korsel-sebut-trump-layak-dapat-nobel-perdamaian-1525103557
Insert berita  Presiden Korsel Sebut Trump Layak Dapat Nobel Perdamaian
masuk
https://ekbis.sindonews.com/read/1302073/32/kuartal-i-2018-telkom-raih-pendapatan-rp32-3-triliun-1525106106
Insert berita  Kuartal I 2018, Telkom Raih Pendapatan Rp32,3 Triliun
masuk
https://ekbis.sindonews.com/read/1302072/178/kuartal-i-2018-aset-bank-dki-capai-rp51-22-triliun-1525105347
Insert berita  Kuartal I 2018, Aset Bank DKI Capai Rp51,22 Triliun
masuk
https://international.sindonews.com/read/1302062/42/trump-ingin-bertemu-jong-un-di-desa-gencatan-senjata-korea-1525103395
Insert berita  Trump Ingin Bertemu Jong

https://daerah.sindonews.com/read/1302033/203/gabung-ke-demokrat-kader-hanura-jabar-disambut-gembira-1525095355
Insert berita  Gabung ke Demokrat, Kader Hanura Jabar Disambut Gembira
masuk
https://ekbis.sindonews.com/read/1302029/33/pemerintah-pastikan-sistem-keuangan-indonesia-stabil-di-kuartal-i-2018-1525094843
Insert berita  Pemerintah Pastikan Sistem Keuangan Indonesia Stabil di Kuartal I 2018
masuk
https://nasional.sindonews.com/read/1302032/13/kpk-resmi-tetapkan-bupati-mojokerto-sebagai-tersangka-1525095213
Insert berita  KPK Resmi Tetapkan Bupati Mojokerto sebagai Tersangka
masuk
https://daerah.sindonews.com/read/1302031/21/penyelundupan-sabu-dalam-gitar-ke-lapas-banceuy-digagalkan-1525095161
Insert berita  Penyelundupan Sabu dalam Gitar ke Lapas Banceuy Digagalkan
masuk
https://lifestyle.sindonews.com/read/1302004/155/kedelai-baik-untuk-cegah-obesitas-ini-penjelasannya-1525089538
Insert berita  Kedelai Baik untuk Cegah Obesitas, Ini Penjelasannya
masuk
https://soccer.sindonews.